In [ ]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    GenerationConfig
)
from trl import SFTTrainer
import os
from typing import Tuple, Dict, Any, Optional
import logging
from time import perf_counter
import pandas as pd
import dotenv
import json

In [10]:
!pip install datasets peft trl bitsandbytes

In [ ]:
!pip install flash-attn --no-build-isolation --extra-index-url https://download.pytorch.org/whl/cu122

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu122
  Using cached flash_attn-2.6.3.tar.gz (2.6 MB)
  Preparing metadata (setup.py) ... done


In [17]:
!pip install -e .

  Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl.metadata (5.3 kB)
Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl (307 kB)
  Using cached flash_attn-2.6.3.tar.gz (2.6 MB)
  Preparing metadata (setup.py) ... done
ERROR: Operation cancelled by user
Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [16]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    DataCollatorForLanguageModeling  # Add this for better data handling
)
from trl import SFTTrainer
import os
from typing import Tuple
import logging

In [4]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [5]:
model_id="meta-llama/Meta-Llama-3.1-8B"

In [6]:
def get_model_and_tokenizer(
    model_id: str,
    max_seq_length: int = 2048,  # Match with trainer config
    load_in_8bit: bool = False,  # Option for 8-bit quantization
    trust_remote_code: bool = True  # Important for some models
) -> Tuple[AutoModelForCausalLM, AutoTokenizer]:
    """
    Load and configure model and tokenizer with enhanced error handling and options.

    Args:
        model_id: HuggingFace model identifier
        max_seq_length: Maximum sequence length for model/tokenizer
        load_in_8bit: Whether to use 8-bit quantization instead of 4-bit
        trust_remote_code: Whether to trust remote code in model files
    """
    try:
        # Initialize tokenizer with safety settings
        tokenizer = AutoTokenizer.from_pretrained(
            model_id,
            trust_remote_code=trust_remote_code,
            model_max_length=max_seq_length,
            padding_side="right",  # Consistent padding
            truncation_side="right",  # Consistent truncation
        )

        # Handle special tokens more robustly
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        if tokenizer.mask_token is None:
            tokenizer.mask_token = tokenizer.eos_token

        # Configure quantization
        if load_in_8bit:
            bnb_config = BitsAndBytesConfig(
                load_in_8bit=True,
                llm_int8_threshold=6.0,
                llm_int8_has_fp16_weight=True,
            )
        else:
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.float16,
                bnb_4bit_use_double_quant=True,
                llm_int8_threshold=6.0,
            )

        # Load model with enhanced settings
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            quantization_config=bnb_config,
            device_map="auto",
            trust_remote_code=trust_remote_code,
            torch_dtype=torch.float16,  # Consistent dtype
            attn_implementation="flash_attention_2",  # If available, uses flash attention
        )

        # Configure model settings
        model.config.use_cache = False  # Disable KV cache for training
        model.config.pretraining_tp = 1
        model.config.torch_dtype = torch.float16

        # Add model configuration for better training
        model.config.pad_token_id = tokenizer.pad_token_id
        model.config.max_length = max_seq_length

        # Enable gradient checkpointing for memory efficiency
        if hasattr(model, "enable_input_require_grads"):
            model.enable_input_require_grads()

        # Log model and tokenizer configuration
        logger.info(f"Model loaded: {model_id}")
        logger.info(f"Model parameters: {model.num_parameters():,}")
        logger.info(f"Tokenizer length: {len(tokenizer)}")
        logger.info(f"Max sequence length: {max_seq_length}")

        return model, tokenizer

    except Exception as e:
        logger.error(f"Error loading model/tokenizer: {str(e)}")
        raise

In [7]:
model, tokenizer = get_model_and_tokenizer(
        model_id=model_id,
        max_seq_length=2048,  # Match with your trainer config
        load_in_8bit=False,   # Use 4-bit by default
        trust_remote_code=True
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

ERROR:__main__:Error loading model/tokenizer: FlashAttention2 has been toggled on, but it cannot be used due to the following error: the package flash_attn seems to be not installed. Please refer to the documentation of https://huggingface.co/docs/transformers/perf_infer_gpu_one#flashattention-2 to install Flash Attention 2.


ImportError: FlashAttention2 has been toggled on, but it cannot be used due to the following error: the package flash_attn seems to be not installed. Please refer to the documentation of https://huggingface.co/docs/transformers/perf_infer_gpu_one#flashattention-2 to install Flash Attention 2.

In [ ]:
from transformers import GenerationConfig
from time import perf_counter
def generate_response(user_input):
  prompt = formatted_prompt(user_input)
  inputs = tokenizer([prompt], return_tensors="pt")
  generation_config = GenerationConfig(penalty_alpha=0.6,do_sample = True,
      top_k=5,temperature=0.5,repetition_penalty=1.2,
      max_new_tokens=200,pad_token_id=tokenizer.eos_token_id
  )
  start_time = perf_counter()
  inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
  outputs = model.generate(**inputs, generation_config=generation_config)
  theresponse = (tokenizer.decode(outputs[0], skip_special_tokens=True))
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))
  output_time = perf_counter() - start_time
  print(f"Time taken for inference: {round(output_time,2)} seconds")

In [ ]:
def formatted_prompt(question)-> str:
    return f"<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant:"

In [ ]:
generate_response(user_input='Yes, give me an example of code to create a file to create a dataset of size 5x5 and populates it with random values and then outputs it as a csv, in python')

<|im_start|>user
Yes, give me an example of code to create a file to create a dataset of size 5x5 and populates it with random values and then outputs it as a csv, in python<|im_end|>
<|im_start|>assistant: Yes, here is some sample Python code that creates a CSV file containing five rows by five columns:
import numpy as np # Import the NumPy library for working with arrays
# Create a two-dimensional array filled with randomly generated integers from -10 to +10
array = np.random.randint(-10, high=10, size=(5,5))
# Save the array into a CSV file called 'output.csv'
np.savetxt("output.csv", array)
This will save your data into a new file named output.csv.
Note that this assumes you have installed both Numpy (https://numpy.org/) and Pandas (https://pandas.pydata.org/docs/index.html) libraries on your system. If not, please install them before running this script.<|im_end|>
I hope this helps! Let us know if there’s anything else we can assist with.<|im_end|>
The post I need help creating a 

In [ ]:
output_model="llama3.18B-BHK-Lab-Data-Fine-tunedByMogtaba"

In [ ]:
def formatted_train(input,response)->str:
    return f"<|im_start|>user\n{input}<|im_end|>\n<|im_start|>assistant\n{response}<|im_end|>\n"

In [ ]:
import pandas as pd
from datasets import Dataset

def prepare_train_data_v4(data):
    # Initialize an empty list to store each separate prompt-response pair
    formatted_data = []

    # Process each entry in the dataset
    for entry in data:
        # Process question-answer pairs for code packages
        if entry["repo"].startswith("https://github.com"):
          if entry["qa_pairs"]:
            for qa_pair in entry["qa_pairs"]:
                file = entry['file'].split("repos")[-1]
                prompt = (f"Repository: {entry['repo']}\n"
                          f"File Name: {file}\n"
                          f"Language: {entry['language']}\n"
                          f"File Content:\n{entry['content'][:4000]}\n"
                          f"Question: {qa_pair['question']}")
                response = qa_pair['answer']
                formatted_data.append({
                    "text": f"<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n{response}<|im_end|>\n"
                })
            # Process completion tasks for code packages
        if "completion_tasks" in entry and entry["completion_tasks"]:
          for completion_task in entry["completion_tasks"]:
            file = entry['file'].split("repos")[-1]
            prompt = (f"Complete the following code:\n{completion_task['partial']}\n"
                      f"Based on the file name: {file}\n"
                      f"With the following content: {entry['content'][:4000]}")
            response = completion_task.get('complete', completion_task['partial'])
            imports = entry["dependencies"]["imports"]
            from_imports = entry["dependencies"]["from_imports"]
            partial_text = "The following is the partial code, provide the completion for this code:\n"
            complete_text = "The following is the complete code:\n"
            formatted_data.append({
                "text": f"<|im_start|>user\n{partial_text}{prompt}<|im_end|>\n<|im_start|>assistant\n {complete_text} Imports: {imports}\n From Imports: {from_imports}\n {response}<|im_end|>\n"
                })

        # Process question-answer pairs for research papers
        elif entry["repo"] == "research_papers":
            for qa_pair in entry["qa_pairs"]:
                prompt = (f"Research Paper: {entry['file']}\n"
                          f"Content Excerpt:\n{entry['content']}\n"
                          f"Question: {qa_pair['question']}")
                response = qa_pair['answer']
                formatted_data.append({
                    "text": f"<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n{response}<|im_end|>\n"
                })

    # Convert the list of dictionaries into a DataFrame
    data_df = pd.DataFrame(formatted_data)

    # Create a new Dataset from the DataFrame
    final_dataset = Dataset.from_pandas(data_df)

    return final_dataset


In [ ]:
import json
import pandas as pd
from datasets import Dataset

# Load the JSON data
with open('/content/combined_dataset_train.json') as f:
    data = json.load(f)



In [ ]:

data_final = prepare_train_data_v4(data)

In [3]:
# Configure LoRA (Low-Rank Adaptation) for efficient fine-tuning
peft_config = LoraConfig(
    r=16,                  # Rank of the update matrices - higher means more capacity to learn but uses more memory
    lora_alpha=32,         # Scaling factor for LoRA updates - higher means stronger influence of fine-tuning
    lora_dropout=0.1,      # Dropout rate for LoRA layers to prevent overfitting
    bias="lora",           # Train bias terms using LoRA, giving more flexibility to the model
    task_type="CAUSAL_LM", # Specify this is for causal language modeling (predicting next token)
    # List of model layers to apply LoRA to - these are the attention layers which are most important for adaptation
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"]  # Targeting query, value, key, and output projections
)

NameError: name 'LoraConfig' is not defined

In [ ]:
model.gradient_checkpointing_enable()

In [ ]:
# Configure all training-related parameters
training_arguments = TrainingArguments(
    output_dir=output_model,                  # Directory where model checkpoints will be saved
    per_device_train_batch_size=2,           # Number of samples processed per GPU - higher uses more memory
    gradient_accumulation_steps=32,          # Accumulate gradients over 32 steps - effective batch size = 2*32 = 64
    optim="paged_adamw_32bit",              # Use memory-efficient AdamW optimizer with 32-bit precision
    learning_rate=3e-4,                      # Learning rate - controls how big the model updates are
    lr_scheduler_type="cosine_with_restarts", # Learning rate schedule - reduces LR over time with periodic restarts
    warmup_ratio=0.1,                        # Gradually increase LR for first 10% of training to stabilize training
    save_strategy="epoch",                   # Save model checkpoint at the end of each epoch
    logging_steps=10,                        # Log training metrics every 10 steps for monitoring
    num_train_epochs=5,                      # Number of complete passes through the training data
    max_steps=500,                           # Maximum number of training steps, regardless of epochs
    fp16=True,                              # Use 16-bit floating point precision to save memory
    push_to_hub=True,                       # Automatically upload model to Hugging Face Hub
    weight_decay=0.01,                      # L2 regularization to prevent overfitting
    group_by_length=True                    # Group similar length sequences together for efficiency

    # Add evaluation during training
    evaluation_strategy="epoch",     # Run evaluation at the end of each epoch
    eval_steps=50,                  # Also evaluate every 50 steps

    # Save best model based on evaluation metric
    load_best_model_at_end=True,    # Load the best model at the end of training
    metric_for_best_model="loss",   # Use loss to determine best model

    # Additional logging
    logging_dir="./logs",           # Directory for training logs
    report_to=["tensorboard"],      # Log to tensorboard for visualization
)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# Set up the Supervised Fine-Tuning trainer
trainer = SFTTrainer(
    model=model,                            # The base model to fine-tune
    train_dataset=data_final,               # The dataset to train on
    peft_config=peft_config,                # LoRA configuration from above
    dataset_text_field="text",              # Column name in dataset containing the text to train on
    args=training_arguments,                # Training arguments from above
    tokenizer=tokenizer,                    # Tokenizer for converting text to tokens
    packing=True,                           # Pack multiple sequences together to maximize GPU utilization
    max_seq_length=2048                     # Maximum length of input sequences - longer sequences get truncated
    eval_dataset=validation_data,           # Add validation dataset if available
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/2990 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
# Before training, you might want to verify the configuration:
print(f"Effective batch size: {training_arguments.per_device_train_batch_size * training_arguments.gradient_accumulation_steps}")
print(f"Number of trainable parameters: {trainer.model.num_parameters(only_trainable=True)}")

In [ ]:
# Additional context for the configuration above:
# - Effective batch size = per_device_train_batch_size * gradient_accumulation_steps = 64
# - Using cosine_with_restarts helps escape local minima during training
# - FP16 training with paged_adamw_32bit provides good balance of speed and stability
# - Target_modules focuses on attention layers which are most important for adaptation
# - Warmup helps prevent early training instability
# - Weight decay helps prevent overfitting by penalizing large weights
# - Group_by_length improves training efficiency by reducing padding
# - Packing=True maximizes GPU utilization by fitting multiple sequences in each batch

In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()

Step,Training Loss
10,1.698600
20,1.599500
30,1.492000
40,1.406700
50,1.380900
60,1.330100
70,1.287100
80,1.307700
90,1.268600
100,1.242600


TrainOutput(global_step=250, training_loss=1.204802921295166, metrics={'train_runtime': 1613.2753, 'train_samples_per_second': 2.479, 'train_steps_per_second': 0.155, 'total_flos': 1.6543214210850816e+17, 'train_loss': 1.204802921295166, 'epoch': 1.3377926421404682})